In [1]:
import gc
gc.collect()

import sys
sys.path.insert(0, '../')
import logging
logging.basicConfig(level=logging.ERROR)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from cryptotrader.exchange_api.poloniex import Poloniex
from cryptotrader.envs.trading import BacktestDataFeed, BacktestEnvironment
from cryptotrader.envs.utils import convert_and_clean, convert_to
from cryptotrader.agents import apriori
from cryptotrader.utils import array_normalize

from time import time, sleep

from bokeh.io import output_notebook
output_notebook()
%matplotlib inline
%load_ext line_profiler

Loading BokehJS ...

In [2]:
# Environment setup
obs_steps = 300
period = 30
pairs = ["USDT_BTC", "USDT_ETH", "USDT_LTC", "USDT_XRP"]
init_funds = {"BTC":'0.00000000', "ETH":'0.00000000', "LTC":'0.00000000', "XRP":'0.00000000', "USDT":'100.00000000'}

tapi = Poloniex()
tapi = BacktestDataFeed(tapi, period, pairs, init_funds)
tapi.download_data(end=datetime.timestamp(datetime.utcnow() - timedelta(days=100)),
                       start=datetime.timestamp(datetime.utcnow() - timedelta(days=150)))

env = BacktestEnvironment(period, obs_steps, tapi, 'backtest_env')
env.add_pairs(pairs)
env.fiat = 'USDT'
obs = env.reset()

INFO:backtest_env:[Trading Environment initialization]
Trading Environment Initialized!



In [3]:
# Train run
# Training params
nb_steps = 100
batch_size = 4
nb_max_episode_steps = 4

agent = apriori.MomentumTrader()

hp = {
    'ma1': [2, env.obs_steps],
    'ma2': [2, env.obs_steps],
    'std_span': [2, env.obs_steps],
    'alpha_up': [1e-4, 5e-1],
    'alpha_down': [1e-4, 5e-1]
    }

search_space = {'mean_type':{'simple': hp,
                             'exp': hp,
                             'kama': hp
                             }
                }
constrains = [lambda mean_type, ma1, ma2, std_span, alpha_up, alpha_down: ma1 < ma2]

opt_params, info = agent.fit(env, nb_steps, batch_size, search_space, constrains, nb_max_episode_steps=nb_max_episode_steps)
print("\n", opt_params,"\n", env.status)

# Previous found params:
# {'mean_type': 'kama', 'alpha_down': 0.4128425807825884, 'alpha_up': 0.37140222586733046, 'ma1': 133, 'ma2': 234, 'std_span': 39}

agent.test(env, verbose=True)
env.plot_results();

Optimizing model for 100 steps with batch size 4...
Optimization step 45/100, step reward: 0.9999039341124829, ETC: 0 days 00:02:04.942052                     
 {'mean_type': 'kama', 'alpha_down': 0.22015884706322197, 'alpha_up': 0.488338909967122, 'ma1': 167.44716360595115, 'ma2': 195.641796875, 'std_span': 143.841015625} 
 {'OOD': False, 'Error': False, 'ValueError': False, 'ActionError': False}
>> step 2099/2099, 100 % done, Cumulative Reward: 0.8754479681585929, ETC: 0 days 00:00:00                                 
################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 -0.000192
Portifolio / Benchmark Sharpe ratio:      -0.168511 / -0.112627
Portifolio / Benchmark Omega ratio:       0.970733 / 0.980044
Portifolio / Benchmark max drawdown:      -1.075676 / -131.588662


In [4]:
# Validation run
tapi.download_data(end=datetime.timestamp(datetime.now() - timedelta(days=50)),
                       start=datetime.timestamp(datetime.now() - timedelta(days=100)))

agent.test(env, verbose=True)
env.plot_results();

>> step 2099/2099, 100 % done, Cumulative Reward: 1.3343646663155542, ETC: 0 days 00:00:00                                 
################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 -0.005526
Portifolio / Benchmark Sharpe ratio:      0.211062 / 0.204748
Portifolio / Benchmark Omega ratio:       1.043290 / 1.041077
Portifolio / Benchmark max drawdown:      -1.062474 / -4.663386


In [5]:
# Test run
tapi.download_data(end=datetime.timestamp(datetime.now()),
                       start=datetime.timestamp(datetime.now() - timedelta(days=50)))

agent.test(env, verbose=True)
env.plot_results();

>> step 2097/2097, 100 % done, Cumulative Reward: 1.135901482297302, ETC: 0 days 00:00:00                                  
################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 -0.025374
Portifolio / Benchmark Sharpe ratio:      0.128993 / 0.262833
Portifolio / Benchmark Omega ratio:       1.023370 / 1.047421
Portifolio / Benchmark max drawdown:      -1.001847 / -1.467730
